In [1]:
import subprocess

# Util functions

In [2]:
def biascorrect(input_img, output_img):
    s1=['N4BiasFieldCorrection','--bspline-fitting','[10,4]','-d','3','--input-image']
    s2=['--convergence', '[150x100x50x30,1e-16]', '--output']
    s3=['--shrink-factor', '2']
    
    cmnd=s1+[input_img]+s2+[output_img]+s3
    result = subprocess.run(cmnd, capture_output=True, text=True)
    print(result.stdout)
    
def apply_transform(input_img, output_img, ref_img, transformation):
    # input_img: animal high_res or axial postsurgery img, pre-registered manually to presurgery space
    # output_img: registered high_res or axial postsurgery img to atlas space
    # ref_img: atlas
    
    s1=['antsApplyTransforms', '--default-value', '0', '--float', '0', '--input']
    s2=['--input-image-type', '0', '--interpolation', 'BSpline[5]', '--output']
    s3=['--reference-image']
    s4=['--transform']
   
    cmnd=s1+[input_img]+s2+[output_img]+s3+[ref_img]+s4+[transformation]
    result = subprocess.run(cmnd, capture_output=True, text=True)
    print(result.stdout)
    
def apply_transform_atlas(input_img, output_img, ref_img, transformation):
    # input_img: atlas
    # output_img: new segmentation specified with animal_id
    # ref_img: unregistered, resampled TurboRARE img of animal_id
    
    s1=['antsApplyTransforms', '--default-value', '0', '--float', '0', '--input']
    s2=['--input-image-type', '0', '--interpolation', 'BSpline[5]', '--output']
    s3=['--reference-image']
    s4=['--transform']
#     s5=["--interpolation", "GenericLabel"]
   
    cmnd=s1+[input_img]+s2+[output_img]+s3+[ref_img]+s4+["["+transformation+", 1]"]
    result = subprocess.run(cmnd, capture_output=True, text=True)
    print(result.stdout)
    
def apply_transform_segmentation(input_img, output_img, ref_img, transformation):
    # input_img: segmentation file from atlas
    # output_img: new segmentation specified with animal_id
    # ref_img: unregistered, resampled TurboRARE img of animal_id
    # transformation: transformation matrix output of ANTs registration
    
    s1=['antsApplyTransforms', '--default-value', '0', '--float', '0', '--input']
    s2=['--input-image-type', '0', '--interpolation', 'BSpline[5]', '--output']
    s3=['--reference-image']
    s4=['--transform']
    s5=["--interpolation", "GenericLabel"]
   
    cmnd=s1+[input_img]+s2+[output_img]+s3+[ref_img]+s4+["["+transformation+", 1]"]+s5
    result = subprocess.run(cmnd, capture_output=True, text=True)
    print(result.stdout)
    

In [56]:
input_img="/Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-postsurgery20240307/anat/sub-rEO_06_ses-postsurgery20240307_task-rest_acq-T2TurboRAREhighRes_T2w-ind_19.nii.gz"
output_img="/Users/mri_registration/SAMRI/samri_output/rEO_06/results/generic_work_20240307-to-presurgery/preopRegged_sub-rEO_06_ses-postsurgery20240307_task-rest_acq-T2TurboRAREhighRes_T2w-ind_19.nii.gz"
ref_img="/Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-presurgery/anat/sub-rEO_06_ses-presurgery_task-rest_acq-TurboRAREregResampled39um_T2w-ind_00.nii.gz"
transformation="/Users/mri_registration/SAMRI/samri_output/rEO_06/results/generic_work_20240307-to-presurgery/_ind_type_52_coronal/s_register/output_Composite.h5"

apply_transform(input_img, output_img, ref_img, transformation)

In [8]:
input_img='/Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_DWI_v1.01.nii.gz'
transformation='/Users/mri_registration/SAMRI/samri_output/rEO_05/results/generic_work/_ind_type_28_affine_good/s_register/output_Composite.h5'
ref_img='/Users/mri_registration/SAMRI/samri_output/rEO_05/bids/sub-rEO_05/ses-presurgery/anat/sub-rEO_05_ses-presurgery_task-rest_acq-TurboRARE_T2w-ind_10_resampled.nii.gz'
output_img='/Users/mri_registration/SAMRI/samri_output/rEO_05/dwiAtlas_affine_eo05_39um.nii.gz'

apply_transform_atlas(input_img, output_img, ref_img, transformation)

In [9]:
seg_file = '/Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_atlas_v4.nii.gz'
out_file = '/Users/mri_registration/SAMRI/samri_output/rEO_05/rEO_05-affine-atlas.nii.gz'

apply_transform_labels(seg_file, out_file, ref_img, transformation)

In [34]:
path="/Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-postsurgery20240307/sub-rEO_06_ses-presurgery_task-rest_acq-TurboRAREreg_T2w-ind_00.nii.gz"

biascorrect(path, path)

In [35]:
s="antsRegistration --float --collapse-output-transforms 1 --dimensionality 3 --initial-moving-transform [ /Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_T2star_v1.01.nii.gz, /Users/mri_registration/SAMRI/samri_output/rEO_05/results/generic_work/_ind_type_849/s_biascorrect/sub-rEO_05_ses-presurgery_task-rest_acq-TurboRAREreg_T2w-ind_10_corrected.nii.gz, 0 ] --initialize-transforms-per-stage 0 --interpolation Linear --output output_ --transform Translation[ 0.1 ] --metric MeanSquares[ /Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_T2star_v1.01.nii.gz, /Users/mri_registration/SAMRI/samri_output/rEO_05/results/generic_work/_ind_type_849/s_biascorrect/sub-rEO_05_ses-presurgery_task-rest_acq-TurboRAREreg_T2w-ind_10_corrected.nii.gz, 1, 32, Regular, 0.3 ] --convergence [ 1000x500x500, 1e-16, 20 ] --smoothing-sigmas 2.0x1.0x0.0vox --shrink-factors 4x2x1 --use-estimate-learning-rate-once 0 --use-histogram-matching 0 --masks [ /Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_v2_brainmask_bin.nii.gz, NULL ] --transform Rigid[ 0.1 ] --metric MI[ /Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_T2star_v1.01.nii.gz, /Users/mri_registration/SAMRI/samri_output/rEO_05/results/generic_work/_ind_type_849/s_biascorrect/sub-rEO_05_ses-presurgery_task-rest_acq-TurboRAREreg_T2w-ind_10_corrected.nii.gz, 1, 32, Regular, 0.25 ] --convergence [ 1000x500x250x100, 1e-06, 10 ] --smoothing-sigmas 3.0x2.0x1.0x0.0vox --shrink-factors 8x4x2x1 --use-estimate-learning-rate-once 0 --use-histogram-matching 1 --masks [ /Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_v2_brainmask_bin.nii.gz, NULL ] --winsorize-image-intensities [ 0.005, 0.995 ]  --write-composite-transform 1"

In [2]:
# s="antsApplyTransforms --default-value 0 --float 0 --input /Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-postsurgery20240201/preopRegged_sub-rEO_06_ses-postsurgery20240201_task-rest_acq-T2TurboRAREhighRes_T2w-ind_6.nii.gz --input-image-type 0 --interpolation BSpline[ 5 ] --output /Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-postsurgery20240201/Regged_sub-rEO_06_ses-postsurgery20240201_task-rest_acq-T2TurboRAREhighRes_T2w-ind_6.nii.gz --reference-image /Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_T2star_v1.01.nii.gz --transform /Users/mri_registration/SAMRI/samri_output/rEO_06/results/generic_work/_ind_type_24/s_register/output_Composite.h5"
# s="N4BiasFieldCorrection --bspline-fitting [ 10, 4 ] -d 3 --input-image /Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-postsurgery20240201/preopRegged_sub-rEO_06_ses-postsurgery20240201_task-axial_acq-T2TurboRARE_T2w-ind_4.nii.gz --convergence [ 150x100x50x30, 1e-16 ] --output /Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-postsurgery20240201/biascorrected-preopRegged_sub-rEO_06_ses-postsurgery20240201_task-axial_acq-T2TurboRARE_T2w-ind_4.nii.gz --shrink-factor 2"